In [530]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import requests
import urllib, json
from PIL import Image
import datetime

In [531]:
# Подключаем api_key из файла
f = open('API_KEY.txt', 'r')
api_key = f.read()
f.close()

In [532]:
# Составляем URL request
api_root = 'https://api.nasa.gov/'
api_product = 'neo/rest/v1/feed?'
start_date = '2020-07-07'
end_date = '2020-07-07'
url = f"{api_root}{api_product}start_date={start_date}&end_date={end_date}&api_key={api_key}"

In [533]:
'''
    Функция навигации по данным из api response
    возвращает ссылку на данные в другом временном отрезке

    step = 'next' -- следующий день от заданного start_date в URL request
    step = 'prev' -- предыдущий день от заданного start_date в URL request
    step = 'self' -- текущий день, равный start_date в URL request
'''

def get_url(url, step):
    url_level = urllib.request.urlopen(url)
    data_level = json.loads(url_level.read())['links']
    if step == 'next':
        return data_level[step]
    elif step == 'prev':
        return data_level[step]
    elif step == 'self':
        return data_level[step]

In [534]:
'''
    Вариант использования бибилотеки datetime для скольжения по датам (не используется в коде)
    и вывод даты в строку

    delta = datetime.timedelta(days=1)
    day_current = datetime.date.fromisoformat(start_date) + delta
    date_str = datetime.date.strftime(day_current, '%Y-%m-%d')
'''

"\n    Вариант использования бибилотеки datetime для скольжения по датам (не используется в коде)\n    и вывод даты в строку\n\n    delta = datetime.timedelta(days=1)\n    day_current = datetime.date.fromisoformat(start_date) + delta\n    date_str = datetime.date.strftime(day_current, '%Y-%m-%d')\n"

In [535]:
# Навигация по ссылкам
# Работаем с данными за один день 2020-07-07
url_level_0 = urllib.request.urlopen(url)
data_level_0 = pd.DataFrame.from_dict(json.loads(url_level_0.read()))
data_level_0

,links,element_count,near_earth_objects
next,http://api.nasa.gov/neo/rest/v1/feed?start_dat...,11,NaN
prev,http://api.nasa.gov/neo/rest/v1/feed?start_dat...,11,NaN
self,http://api.nasa.gov/neo/rest/v1/feed?start_dat...,11,NaN
2020-07-07,NaN,11,[{'links': {'self': 'http://api.nasa.gov/neo/r...


In [536]:
# Выбираем один элемент из списка за один день
data_day = data_level_0.near_earth_objects[3]
data_day_item = data_day[0]
data_day_item

{'links': {'self': 'http://api.nasa.gov/neo/rest/v1/neo/3343102?api_key=8Mbg6rxSagXNYxrUbC8ynidJUcLn9h4CC7ucuT21'},
 'id': '3343102',
 'neo_reference_id': '3343102',
 'name': '(2006 SS19)',
 'nasa_jpl_url': 'http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3343102',
 'absolute_magnitude_h': 18.9,
 'estimated_diameter': {'kilometers': {'estimated_diameter_min': 0.4411182,
   'estimated_diameter_max': 0.9863702813},
  'meters': {'estimated_diameter_min': 441.1181999969,
   'estimated_diameter_max': 986.3702813054},
  'miles': {'estimated_diameter_min': 0.2740980571,
   'estimated_diameter_max': 0.6129018881},
  'feet': {'estimated_diameter_min': 1447.2382352778,
   'estimated_diameter_max': 3236.1230737181}},
 'is_potentially_hazardous_asteroid': False,
 'close_approach_data': [{'close_approach_date': '2020-07-07',
   'close_approach_date_full': '2020-Jul-07 08:13',
   'epoch_date_close_approach': 1594109580000,
   'relative_velocity': {'kilometers_per_second': '11.947301649',
    'kilometers_per_h

In [537]:
# Создаем словари с выбранными данными для DataFrame
dict_estimated_diameter = data_day_item['estimated_diameter']['kilometers']
dict_close_approach_data = data_day_item['close_approach_data'][0]
dict_relative_velocity = dict_close_approach_data['relative_velocity']
dict_miss_distance = dict_close_approach_data['miss_distance']

In [538]:
# Чистим словарь от вложенных словарей
keys = ['links','estimated_diameter','close_approach_data', 'relative_velocity', 'miss_distance']

for key in keys:
    data_day_item.pop(key, None)
data_day_item_short = data_day_item
data_day_item_short

{'id': '3343102',
 'neo_reference_id': '3343102',
 'name': '(2006 SS19)',
 'nasa_jpl_url': 'http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3343102',
 'absolute_magnitude_h': 18.9,
 'is_potentially_hazardous_asteroid': False,
 'is_sentry_object': False}

In [539]:
# Создаем DataFrame
# Объединяем DataFrame
df1 = pd.DataFrame([data_day_item_short])
df2 = pd.DataFrame(dict_estimated_diameter, index=[0]).reset_index(drop=True)
df3 = pd.DataFrame(dict_close_approach_data, index=[0]).reset_index(drop=True)
df4 = pd.DataFrame(dict_relative_velocity, index=[0]).reset_index(drop=True)
df5 = pd.DataFrame(dict_miss_distance, index=[0]).reset_index(drop=True)
df_item = df1.join(df2).join(df3).join(df4).join(df5)
df_item

,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_min,estimated_diameter_max,close_approach_date,...,relative_velocity,miss_distance,orbiting_body,kilometers_per_second,kilometers_per_hour,miles_per_hour,astronomical,lunar,kilometers,miles
0,3343102,3343102,(2006 SS19),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3343102,18.9,False,False,0.441118,0.98637,2020-07-07,...,NaN,NaN,Earth,11.947301649,43010.285936445,26724.9190586761,0.4460874678,173.5280249742,66733735.016573586,41466420.1158480468


In [540]:
# Чистим DataFrame от нерелевантных данных, переименовываем
df_item = df1.join(df2).join(df3).join(df4).join(df5).drop(['nasa_jpl_url',
                                                'neo_reference_id',
                                                'miss_distance',
                                                'miles_per_hour',
                                                'astronomical',
                                                'lunar',
                                                'miles'
                                                ], axis=1
).rename(columns={'kilometers': 'miss_distance_kilometers',
                'estimated_diameter_min': 'estimated_diameter_km_min',
                'estimated_diameter_max': 'estimated_diameter_km_max'})
df_item.to_dict()

{'id': {0: '3343102'},
 'name': {0: '(2006 SS19)'},
 'absolute_magnitude_h': {0: 18.9},
 'is_potentially_hazardous_asteroid': {0: False},
 'is_sentry_object': {0: False},
 'estimated_diameter_km_min': {0: 0.4411182},
 'estimated_diameter_km_max': {0: 0.9863702813},
 'close_approach_date': {0: '2020-07-07'},
 'close_approach_date_full': {0: '2020-Jul-07 08:13'},
 'epoch_date_close_approach': {0: 1594109580000},
 'relative_velocity': {0: nan},
 'orbiting_body': {0: 'Earth'},
 'kilometers_per_second': {0: '11.947301649'},
 'kilometers_per_hour': {0: '43010.285936445'},
 'miss_distance_kilometers': {0: '66733735.016573586'}}

In [545]:
# Делаем список данных за n дней

items_list = []
keys = ['links','estimated_diameter','close_approach_data', 'relative_velocity', 'miss_distance']


# ТЕСТ!!!!!
keys_list = ['id',
 'name',
 'absolute_magnitude_h',
 'is_potentially_hazardous_asteroid',
 'is_sentry_object',
 'estimated_diameter_km_min',
 'estimated_diameter_km_max',
 'close_approach_date',
 'close_approach_date_full',
 'epoch_date_close_approach',
 'relative_velocity',
 'orbiting_body',
 'kilometers_per_second',
 'kilometers_per_hour',
 'miss_distance_kilometers']

dff = pd.DataFrame(columns=keys_list)

# n - количество запросов(дней)
n = 10

for i in range(1, n+1):
    current = get_url(url, 'next')
    url = current
    
    # Навигация по ссылкам, работаем с данными за один день
    url_level_0 = urllib.request.urlopen(url)
    data_level_0 = pd.DataFrame.from_dict(json.loads(url_level_0.read()))
    
    # Cписок элементов за один день
    data_days = data_level_0.near_earth_objects[3]

    # Цикл по элементам списка за один день
    for data_day_item in data_days:
        dict_estimated_diameter = data_day_item['estimated_diameter']['kilometers']
        dict_close_approach_data = data_day_item['close_approach_data'][0]
        dict_relative_velocity = dict_close_approach_data['relative_velocity']
        dict_miss_distance = dict_close_approach_data['miss_distance']

        # Чистим словарь от вложенных словарей
        for key in keys:
            data_day_item.pop(key, None)
        data_day_item_short = data_day_item

        df1 = pd.DataFrame([data_day_item_short])
        df2 = pd.DataFrame(dict_estimated_diameter, index=[0]).reset_index(drop=True)
        df3 = pd.DataFrame(dict_close_approach_data, index=[0]).reset_index(drop=True)
        df4 = pd.DataFrame(dict_relative_velocity, index=[0]).reset_index(drop=True)
        df5 = pd.DataFrame(dict_miss_distance, index=[0]).reset_index(drop=True)

        df_item = df1.join(df2).join(df3).join(df4).join(df5).drop(['nasa_jpl_url',
                                                        'neo_reference_id',
                                                        'miss_distance',
                                                        'miles_per_hour',
                                                        'astronomical',
                                                        'lunar',
                                                        'miles'
                                                        ], axis=1
        ).rename(columns={'kilometers': 'miss_distance_kilometers',
                        'estimated_diameter_min': 'estimated_diameter_km_min',
                        'estimated_diameter_max': 'estimated_diameter_km_max'})
        
        dff = pd.concat([dff, df_item])
df_final = dff.head(15).reset_index()
del df_final['index']
df_final

,id,name,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_km_min,estimated_diameter_km_max,close_approach_date,close_approach_date_full,epoch_date_close_approach,relative_velocity,orbiting_body,kilometers_per_second,kilometers_per_hour,miss_distance_kilometers
0,2137799,137799 (1999 YB),18.61,False,False,0.504144,1.127301,2020-08-09,2020-Aug-09 23:34,1597016040000,NaN,Earth,3.807816871,13708.1407357702,51065828.159464943
1,2162080,162080 (1998 DG16),19.95,False,False,0.271991,0.608191,2020-08-09,2020-Aug-09 02:51,1596941460000,NaN,Earth,18.1840361586,65462.5301709744,65157836.496885616
2,2376788,376788 (2000 RV37),19.55,False,False,0.327005,0.731206,2020-08-09,2020-Aug-09 00:10,1596931800000,NaN,Earth,27.6424399159,99512.7836970924,61407552.41439032
3,2380128,380128 (1997 WB21),20.27,False,False,0.234723,0.524856,2020-08-09,2020-Aug-09 20:21,1597004460000,NaN,Earth,8.700925772,31323.3327793276,18152777.174221057
4,3134264,(2002 RR25),21.09,False,False,0.160900,0.359782,2020-08-09,2020-Aug-09 12:05,1596974700000,NaN,Earth,9.0905959784,32726.145522144,42689629.927044666
5,3524069,(2010 JV153),19.60,False,False,0.319562,0.714562,2020-08-09,2020-Aug-09 08:28,1596961680000,NaN,Earth,6.0899000026,21923.6400094582,32811214.314651779
6,3605257,(2012 KT42),29.00,False,False,0.004213,0.009420,2020-08-09,2020-Aug-09 00:01,1596931260000,NaN,Earth,14.3874507695,51794.8227702914,44621433.567774792
7,3621604,(2012 YK),23.00,False,False,0.066766,0.149293,2020-08-09,2020-Aug-09 06:48,1596955680000,NaN,Earth,15.1525586786,54549.2112430543,70555759.131622057
8,3674789,(2014 NE64),18.88,False,False,0.445200,0.995497,2020-08-09,2020-Aug-09 21:19,1597007940000,NaN,Earth,20.4732281881,73703.6214771618,68159282.650423124
9,3678611,(2014 OA339),24.30,False,False,0.036691,0.082043,2020-08-09,2020-Aug-09 08:18,1596961080000,NaN,Earth,11.6384150685,41898.2942464434,10709420.24624711
